In [317]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl

In [318]:
# get data
dfNFL = nfl.import_pbp_data(years = list(range(1999, 2025, 1)), downcast = True, cache = False, alt_path = None)

1999 done.
2000 done.
2001 done.
2002 done.
2003 done.
2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
Downcasting floats.


In [319]:
list(dfNFL.columns)

['play_id',
 'game_id',
 'old_game_id',
 'home_team',
 'away_team',
 'season_type',
 'week',
 'posteam',
 'posteam_type',
 'defteam',
 'side_of_field',
 'yardline_100',
 'game_date',
 'quarter_seconds_remaining',
 'half_seconds_remaining',
 'game_seconds_remaining',
 'game_half',
 'quarter_end',
 'drive',
 'sp',
 'qtr',
 'down',
 'goal_to_go',
 'time',
 'yrdln',
 'ydstogo',
 'ydsnet',
 'desc',
 'play_type',
 'yards_gained',
 'shotgun',
 'no_huddle',
 'qb_dropback',
 'qb_kneel',
 'qb_spike',
 'qb_scramble',
 'pass_length',
 'pass_location',
 'air_yards',
 'yards_after_catch',
 'run_location',
 'run_gap',
 'field_goal_result',
 'kick_distance',
 'extra_point_result',
 'two_point_conv_result',
 'home_timeouts_remaining',
 'away_timeouts_remaining',
 'timeout',
 'timeout_team',
 'td_team',
 'td_player_name',
 'td_player_id',
 'posteam_timeouts_remaining',
 'defteam_timeouts_remaining',
 'total_home_score',
 'total_away_score',
 'posteam_score',
 'defteam_score',
 'score_differential',
 'po

In [320]:
dfNFL.head(50)

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tack

In [321]:
# sort by game and drive
dfNFL = dfNFL.sort_values(['game_id', 'drive'])

# change of possession variable
dfNFL['change_of_possession'] = (dfNFL['fixed_drive'] != dfNFL['fixed_drive'].shift(1)).astype(int)

# correct start transition variable
dfNFL['end_of_previous_drive'] = (
    dfNFL.groupby('game_id')['fixed_drive_result']
      .shift(1)
)

# Explicitly handle start of game
dfNFL.loc[
    dfNFL.groupby('game_id').head(1).index,
    'end_of_previous_drive'
] = None

/var/folders/6z/0pv81j2524v8c80qsmnrbmcm0000gn/T/ipykernel_241/1558442355.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfNFL['change_of_possession'] = (dfNFL['fixed_drive'] != dfNFL['fixed_drive'].shift(1)).astype(int)
/var/folders/6z/0pv81j2524v8c80qsmnrbmcm0000gn/T/ipykernel_241/1558442355.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfNFL['end_of_previous_drive'] = (


In [322]:
# variable for first drive
dfNFL['first_drive'] = (
    dfNFL['drive'] ==
    dfNFL.groupby(['game_id', 'posteam'])['drive'].transform('min')
).astype(int)

In [323]:
# variable for 3+ score game
dfNFL['ever_3_score'] = (
    dfNFL.groupby('game_id')['score_differential']
         .transform('max')
         .gt(17)
         .astype(int)
)       

In [324]:
# identify 4th downs
dfNFL['is_4th'] = (dfNFL['down'] == 4).astype(int)

# 4th down attempt: went for it (converted or failed)
dfNFL['fourth_attempt'] = (
    dfNFL['fourth_down_converted'].astype(bool) |
    dfNFL['fourth_down_failed'].astype(bool)
).astype(int)

# successful conversion
dfNFL['fourth_converted'] = dfNFL['fourth_down_converted'].fillna(0).astype(int)

# yards to go
dfNFL['ydstogo_all_4th'] = dfNFL['ydstogo'].where(dfNFL['is_4th'] == 1)
dfNFL['ydstogo_attempted'] = dfNFL['ydstogo'].where(dfNFL['fourth_attempt'] == 1)
dfNFL['ydstogo_converted'] = dfNFL['ydstogo'].where(dfNFL['fourth_converted'] == 1)

# score differential
dfNFL['score_diff_all_4th'] = dfNFL['score_differential'].where(dfNFL['is_4th'] == 1)
dfNFL['score_diff_attempted'] = dfNFL['score_differential'].where(dfNFL['fourth_attempt'] == 1)
dfNFL['score_diff_converted'] = dfNFL['score_differential'].where(dfNFL['fourth_converted'] == 1)

fourth_summary = (
    dfNFL
    .groupby(['game_id', 'posteam'])
    .agg(
        # counts
        fourth_downs=('is_4th', 'sum'),
        fourth_attempts=('fourth_attempt', 'sum'),
        fourth_converted=('fourth_converted', 'sum'),
        fourth_failed=('fourth_down_failed', 'sum'),

        # averages: yards to go
        avg_ydstogo_all_4th=('ydstogo_all_4th', 'mean'),
        avg_ydstogo_attempted=('ydstogo_attempted', 'mean'),
        avg_ydstogo_converted=('ydstogo_converted', 'mean'),

        # averages: score differential
        avg_score_diff_all_4th=('score_diff_all_4th', 'mean'),
        avg_score_diff_attempted=('score_diff_attempted', 'mean'),
        avg_score_diff_converted=('score_diff_converted', 'mean'),
    )
    .reset_index()
)

# get rates
fourth_summary['fourth_conversion_rate'] = (
    fourth_summary['fourth_converted'] /
    fourth_summary['fourth_attempts']
)

fourth_summary['fourth_attempt_rate'] = (
    fourth_summary['fourth_attempts'] /
    fourth_summary['fourth_downs']
)

fourth_summary[['fourth_conversion_rate', 'fourth_attempt_rate']] = (
    fourth_summary[['fourth_conversion_rate', 'fourth_attempt_rate']]
    .fillna(0)
)

dfNFL = dfNFL.drop(['fourth_converted', 'fourth_down_failed'], axis = 1)

# merge back
dfNFL = dfNFL.merge(
    fourth_summary,
    on=['game_id', 'posteam'],
    how='left'
)

In [325]:
# first quarter 4th down attempt and conversion
dfNFL['q1_fourth_attempt'] = (
    (dfNFL['qtr'] == 1) &
    (dfNFL['down'] == 4) &
    (dfNFL['fourth_attempt'] == 1)
).astype(int)

dfNFL['q1_fourth_converted'] = (
    (dfNFL['qtr'] == 1) &
    (dfNFL['down'] == 4) &
    (dfNFL['fourth_down_converted'].astype(bool))
).astype(int)

# first drive 4th down attempt and conversion
dfNFL['fd_fourth_attempt'] = (
    (dfNFL['first_drive'] == 1) &
    (dfNFL['down'] == 4) &
    (dfNFL['fourth_attempt'] == 1)
).astype(int)

dfNFL['fd_fourth_converted'] = (
    (dfNFL['first_drive'] == 1) &
    (dfNFL['down'] == 4) &
    (dfNFL['fourth_down_converted'].astype(bool))
).astype(int)

# aggregate early 4th
early_fourth_summary = (
    dfNFL
    .groupby(['game_id', 'posteam'])
    .agg(
        q1_fourth_attempt=('q1_fourth_attempt', 'max'),
        q1_fourth_converted=('q1_fourth_converted', 'max'),
        fd_fourth_attempt=('fd_fourth_attempt', 'max'),
        fd_fourth_converted=('fd_fourth_converted', 'max'),
    )
    .reset_index()
)

dfNFL = dfNFL.drop(['q1_fourth_attempt', 'q1_fourth_converted', 'fd_fourth_attempt', 'fd_fourth_converted'], axis = 1)

# merge back
dfNFL = dfNFL.merge(
    early_fourth_summary,
    on=['game_id', 'posteam'],
    how='left'
)

/var/folders/6z/0pv81j2524v8c80qsmnrbmcm0000gn/T/ipykernel_241/3156422734.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfNFL['q1_fourth_attempt'] = (
/var/folders/6z/0pv81j2524v8c80qsmnrbmcm0000gn/T/ipykernel_241/3156422734.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfNFL['q1_fourth_converted'] = (
/var/folders/6z/0pv81j2524v8c80qsmnrbmcm0000gn/T/ipykernel_241/3156422734.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

In [326]:
# first quarter 2 point attempt and conversion
dfNFL['q1_2pt_attempt'] = (
    (dfNFL['qtr'] == 1) &
    (~dfNFL['two_point_conv_result'].isna())
).astype(int)

dfNFL['q1_2pt_converted'] = (
    (dfNFL['qtr'] == 1) &
    (dfNFL['two_point_conv_result'] == 'success')
).astype(int)

# first drive 2 point attempt and conversion
dfNFL['fd_2pt_attempt'] = (
    (dfNFL['first_drive'] == 1) &
    (~dfNFL['two_point_conv_result'].isna())
).astype(int)

dfNFL['fd_2pt_converted'] = (
    (dfNFL['first_drive'] == 1) &
    (dfNFL['two_point_conv_result'] == 'success')
).astype(int)

# aggregate early 2pt
early_2pt_summary = (
    dfNFL
    .groupby(['game_id', 'posteam'])
    .agg(
        q1_2pt_attempt=('q1_2pt_attempt', 'max'),
        q1_2pt_converted=('q1_2pt_converted', 'max'),
        fd_2pt_attempt=('fd_2pt_attempt', 'max'),
        fd_2pt_converted=('fd_2pt_converted', 'max'),
    )
    .reset_index()
)

dfNFL = dfNFL.drop(['q1_2pt_attempt', 'q1_2pt_converted', 'fd_2pt_attempt', 'fd_2pt_converted'], axis = 1)

# merge back
dfNFL = dfNFL.merge(
    early_2pt_summary,
    on=['game_id', 'posteam'],
    how='left'
)

/var/folders/6z/0pv81j2524v8c80qsmnrbmcm0000gn/T/ipykernel_241/2408231599.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfNFL['q1_2pt_attempt'] = (
/var/folders/6z/0pv81j2524v8c80qsmnrbmcm0000gn/T/ipykernel_241/2408231599.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfNFL['q1_2pt_converted'] = (
/var/folders/6z/0pv81j2524v8c80qsmnrbmcm0000gn/T/ipykernel_241/2408231599.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

In [327]:
# get pass rates entire game
dfTemp = dfNFL[dfNFL['play_type'].isin(['pass', 'run', 'qb_kneel', 'qb_spike'])]

rates = (
    dfTemp
    .groupby(['game_id', 'posteam_type'])['qb_dropback']
    .mean()
    .unstack('posteam_type')      # columns: 'home', 'away'
    .rename(columns={'home': 'home_pass_rate',
                     'away': 'away_pass_rate'})
)

# merge back onto the original df by game_id
dfNFL = dfNFL.merge(rates, on='game_id', how='left')

In [328]:
# get main qb
qb_plays = dfNFL[dfNFL['qb_dropback'] == 1]

qb_map = (
    qb_plays[qb_plays['passer_player_name'].notna()]
    .groupby(['game_id', 'posteam_type'])['passer_player_name']
    .apply(lambda x: x.mode()[0])  # most common QB during dropbacks
    .unstack('posteam_type')       # creates only 'home' and 'away' columns
    .rename(columns={'home': 'home_qb', 'away': 'away_qb'})
)

dfNFL = dfNFL.merge(qb_map, on='game_id', how='left')

In [329]:
# get qb change
qb_plays = dfNFL[dfNFL['qb_dropback'] == 1]

# how many dropbacks each passer had for each game & team
qb_dropback_counts = (
    qb_plays
    .groupby(['game_id', 'posteam_type', 'passer_player_name'])
    .size()
    .reset_index(name='num_dropbacks')
)

min_dropbacks = 3  # tweak this as you like

qb_main = qb_dropback_counts[qb_dropback_counts['num_dropbacks'] >= min_dropbacks]

qb_counts = (
    qb_main
    .groupby(['game_id', 'posteam_type'])['passer_player_name']
    .nunique()
    .unstack('posteam_type')
    .fillna(0)
    .rename(columns={'home': 'home_qb_count', 'away': 'away_qb_count'})
)

qb_counts['home_qb_change'] = (qb_counts['home_qb_count'] > 1).astype(int)
qb_counts['away_qb_change'] = (qb_counts['away_qb_count'] > 1).astype(int)

dfNFL = dfNFL.merge(
    qb_counts[['home_qb_change', 'away_qb_change']],
    on='game_id',
    how='left'
)

In [330]:
# get individual drive passing rates
dfTemp = dfNFL[dfNFL['play_type'].isin(['pass', 'run', 'qb_kneel', 'qb_spike'])]

drive_rates = (
    dfTemp
    .groupby(['game_id', 'drive', 'posteam_type'])['qb_dropback']
    .mean()
    .unstack('posteam_type')   # columns: 'home', 'away'
    .rename(columns={
        'home': 'home_pass_rate_drive',
        'away': 'away_pass_rate_drive'
    })
    .reset_index()
)

dfNFL = dfNFL.merge(drive_rates, on=['game_id', 'drive'], how='left')

In [331]:
# get score at end of drive
drive_scores = (
    dfNFL
    .groupby(['game_id', 'drive'])
    .agg(
        drive_end_score=('score_differential', 'last')
    )
    .reset_index()
)

# get previous drive score differential
drive_scores['prev_drive_score'] = (
    drive_scores
    .groupby('game_id')['drive_end_score']
    .shift(1)
)

# merge back
dfNFL = dfNFL.merge(
    drive_scores[['game_id', 'drive', 'prev_drive_score']],
    on=['game_id', 'drive'],
    how='left'
)

In [332]:
dfNFL_drives = dfNFL[dfNFL['change_of_possession'] == 1]

In [333]:
pd.set_option('display.max_columns', None)
dfNFL_drives.tail()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assi

In [334]:
dfNFL_drives.shape

(171796, 438)

In [335]:
dfNFL_drives.to_csv('/Users/austincoffelt/NFL-Project/data/nfl_drive.csv')